In [62]:
import pandas as pd
df1=pd.read_csv('C://Users//kk//Downloads//SHNDOnlyRad.csv')

In [63]:
df1.head(5)

,Unnamed: 0,Unnamed: 0.1,CATEGORY,TEXT
0,657357,657357,Radiology,"[**Last Name (LF) **],[**First Name3 (LF) **] ..."
1,728603,728603,Radiology,[**2153-1-12**] 10:03 PM\n CHEST (PORTABLE AP)...
2,728604,728604,Radiology,[**2145-3-13**] 2:58 AM\n CT ABDOMEN W/CONTRAS...
3,728605,728605,Radiology,[**2119-3-11**] 11:46 AM\n CT HEAD W/O CONTRAS...
4,732765,732765,Radiology,[**2152-7-18**] 1:06 PM\n CHEST (PORTABLE AP) ...


In [ ]:
df2=pd.DataFrame(data=list(df1['TEXT']),columns=['Text'])
df2.head(5)

In [ ]:
#extract headings
import regex as re
def list_elements():
    list1=[]
    for i in range(len(df2['Text'])):
        x1=df2['Text'][i]
        list1.append(x1)
    return list1
def get_headings(list1):
    extra_list=[]
    for i in range(len(list1)):
        x1=list1[i]
        x1=x1.replace('\n',' ')
        new=re.findall(r'[A-Za-z]+:',x1)
        extra_list.append(new)
    list04=[x1.lower() for x2 in extra_list for x1 in x2]
    final_head=set(list04)
    final_head=list(final_head)
    return final_head
new_list=list_elements()
headings=get_headings(new_list)

In [ ]:
re3='i[A-Za-z]+ss[A-Za-z]+:'
findings_impression=[x1[0] for x1 in [re.findall(re3,i) for i in headings] if x1 != []]

In [ ]:
len(findings_impression),len(findings_vocab)
#... so these are number of words related to findings and impression

In [ ]:
find_list=[i for i in headings if i.startswith('fi')]
re2=r'fi[A-Za-z]+ng[A-Za-z]+:'
findings_vocab=[x1[0] for x1 in [re.findall(re2,i) for i in find_list] if x1 != []]

In [ ]:
if 'findings:' in findings_vocab:
    print(True)

In [34]:
# lets check if a sentence contains a double findings and impression scenario
def prepare_text_data(x):
    x=x.replace('\n',' ')
    x=x.lower()
    return x
x1=pd.DataFrame(data=[*range(0,len(df2['Text']))])
df3=pd.concat([df2,x1],axis=1,join='outer')
df3.rename(columns={'TEXT':'Text',0:'Original_position'},inplace=True)
df3['cleared']=df3['Text'].apply(prepare_text_data)

In [35]:
def findings_length():
    list2=[]
    regex1=r'findings:'
    for c,i in enumerate(df3['cleared']):
        x1=re.findall(regex1,i)
        if len(x1)>=2: 
            list2.append([c,len(x1)])
    return list2
def impression_length():
    list2=[]
    regex1=r'impression:'
    for c,i in enumerate(df3['cleared']):
        x1=re.findall(regex1,i)
        if len(x1)>=2: 
            list2.append([c,len(x1)])
    return list2
length_double_findings=findings_length()
length_double_impression=impression_length()

In [36]:
len(length_double_findings),len(length_double_impression)

(2355, 2811)

In [37]:
findings_length_index=[i[0] for i in length_double_findings]
impression_length_index=[i[0] for i in length_double_impression]

In [38]:
final_double_list=findings_length_index.copy()
final_double_list.extend(impression_length_index)
final_double_list_new=list(set(final_double_list))
print(f'The findings and impression without set double are {len(final_double_list)} \
and with applying set {len(final_double_list_new)}')

The findings and impression without set double are 5166 and with applying set 4438


In [39]:
df4=df3.copy()
df4.drop(labels=final_double_list_new,inplace=True,axis=0)

In [40]:
print(f'Original Dataframe size is {df1.shape} and Reduced dataframe length is {df4.shape}')

Original Dataframe size is (522279, 4) and Reduced dataframe length is (517841, 3)


In [41]:
def seperation(one_string):
    for x in re.finditer(r'findings:',one_string):
        return ((x.start(), x.end()))
df4['finding_list_Position']=df4['cleared'].apply(seperation)

In [42]:
df4.dropna(inplace=True)

In [43]:
df5=df4.copy()

In [ ]:
df5.reset_index(inplace=True)
df5.drop(labels='index',axis=1,inplace=True)
df5

In [ ]:
print(df5['Text'][2])

In [46]:
def get_finding_text(df5):
    list1=[]
    for values,comp in enumerate(df5['cleared']):
        x2=df5['finding_list_Position'][values][0]
        list1.append(comp[x2+10:])
    return list1
clear_findings=get_finding_text(df5)

In [47]:
#
if 'impression:' in clear_findings[0].split(' '):
    print(True)
if 'impression:' in headings:
    print(False)

True
False


In [48]:
# lets use some other metric to extract findings frtom data
def check_impression(clear_findings):
    end=[]
    for string_number,i1 in enumerate(clear_findings):
        x=i1.split(' ')
        for x1 in x: 
            if x1 in headings:
                end.append(f"word is {x1} and index is {x.index(x1)} for string {string_number}")
                break
    return end
end_Findings=check_impression(clear_findings)

In [ ]:
# interpretation
clear_findings[1]

In [50]:
list99=[i.split(' ')[2]  for i in end_Findings]
list98=list(filter(lambda x : x=='impression:',list99))
len(list98),len(list99)
# so 146387 reports have consecutive findings and impression headings

(146837, 180617)

In [51]:
#some exceptions like 

In [52]:
check_existence=[f'{list99.count(i1)} for word {i1}' for i1 in list(set(list99))]
new_existence=[(int(i2.split(' ')[0]),i2.split(' ')[3]) for i2 in [f'{list99.count(i1)} for word {i1}' for i1 in list(set(list99))]]
sorting_ex=sorted(list(filter(lambda x: x[0]>15,new_existence)),key=lambda x : x[0],reverse=True)
list_rm_headings=[i[0] for i in sorting_ex]

In [ ]:
print(sorting_ex)
print(list_rm_headings)
# so why i am doing this because i want to remove such headings which have affectance on extracting impression

In [54]:
values_only_impression=list(filter(lambda x:x[0]=='impression:',[(i1.split(' ')[2],int(i1.split(' ')[6]),int(i1.split(' ')[9])) for i1 in end_Findings]))
consecutive=[clear_findings[value[2]] for value in values_only_impression]

In [55]:
def extract_findings(findings):
    get_only_findings=[]
    for finding in findings:
        result=re.split(r'[A-Za-z]+:',finding)[0].strip()
        get_only_findings.append(result)
    return get_only_findings
findings=extract_findings(consecutive)
                                                         #...Note...........
# In order to compare these extracted findings with original findings use df5 dataframe to compare
# and in order to compare you need values_only_impression list --> and the value at third index is same as df5
def extract_impression(findings):
    #get_only_impression=[]
    #for finding in findings:
        #result=re.split(r'[A-Za-z]+:',finding)[1].strip()
    result=[re.split(r'[A-Za-z]+:',finding)[1].strip() for finding in findings]
    final_result=[re.split(r'[A-Za-z]+:',impression)[0].strip() for impression in result]
        #get_only_impression.append(result)
    return final_result
impressions=extract_impression(consecutive)

In [56]:
len(impressions),len(findings)

(146837, 146837)

In [57]:
data_cleaned=pd.DataFrame(data={'findings':findings,'impressions':impressions})

In [58]:
def everything_over(x):
    get_list=x.split(' ')
    if '(over)' in get_list:
        text=' '.join([get_list[:get_list.index(i1)]  for i1 in x.split(' ') if i1=='(over)'][0])
        return text.strip()
    else:
        return x.strip()
data_cleaned['impressions']=data_cleaned['impressions'].apply(everything_over)
#va1=everything_over(str(data_cleaned['impressions'].iloc[-3]))

In [59]:
data_cleaned['impressions'].iloc[75654]

'1.  new placement of aortic balloon pump and swan-ganz catheter.  2.  no change in left lower lobe and right perihilar consolidation.'

In [60]:
def length_check(x):
    len1=len(x.split(' '))
    return len1
results=[data_cleaned[i1].apply(length_check) for i1 in list(data_cleaned.columns)]
findings_length,impression_length=list(results[0]),list(results[1])
data2=pd.DataFrame(data={'findings_length':findings_length,'impression_length':impression_length})
fin=pd.concat([data_cleaned,data2],axis=1)
index_to_remove=list(fin[fin['impression_length']>=fin['findings_length']].index)
fin.drop(index=index_to_remove,inplace=True)
final_data=fin.copy()

In [100]:
final_data=fin.copy()

In [102]:
final_data["half_length"]=final_data["findings_length"].apply(lambda x : int(x/2))

In [ ]:
final_data[final_data["half_length"]<=final_data["impression_length"]]

In [81]:
#final_data[(final_data["half_length"]>=final_data["findings_length"]) | (final_data["findings_length"]>100)]

In [108]:
#lets make an assumption to only including findings with length almost 100
final_data_reports=final_data[final_data["findings_length"]<=100]

In [113]:
final_data_reports=final_data_reports[final_data_reports["half_length"]>=final_data_reports["impression_length"]]

In [112]:
#final_data_reports[final_data_reports["impression_length"]>=final_data_reports["half_length"]]

In [86]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kk\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [114]:
import regex as re
from nltk.corpus import stopwords 
stop_words = set(stopwords.words('english')) 
def clean_data(list1,name,new_list1):
    for i in list1:
        i=str(i)
        new_string = re.sub(r'[^\w\s]', '', i)
        pattern = r'[0-9]'
        new_string1 = re.sub(pattern, '', new_string)
        new_string2 = re.sub(r'\([^)]*\)', '', new_string1)
        new_string1=new_string2.lower()
        if name=='findings':
            tokens = [w for w in new_string1.split() if not w in stop_words]
            tokens=' '.join(tokens)
            new_list1.append(tokens)
        elif name=='impressions':
            new_list1.append(new_string1)
    return new_list1
x1=[]
list1=clean_data(list(final_data_reports['findings']),final_data_reports['findings'].name,x1)

In [115]:
x2=[]
list2=clean_data(list(final_data_reports['impressions']),final_data_reports['impressions'].name,x2)

In [116]:
def clean_further(list1):
    list5=[]
    for i in list1:
        i=str(i)
        list1=i.split(' ')
        for i in range(list1.count('')):
            list1.remove('')
        x1=' '.join(list1)
        list5.append(x1)
    return list5
list6=clean_further(list1)

In [117]:
list7=clean_further(list2)

In [118]:
data_for_training=pd.DataFrame({'findings':list6,
                'impression':list7})

In [120]:
data_for_training
data_for_training.to_csv("C://Users//kk//Downloads//Assumption_Based.csv")

In [59]:
import pandas as pd
data_for_training=pd.read_csv("C://Users//kk//Downloads//Assumption_Based.csv")

In [60]:
data_for_training

,Unnamed: 0,findings,impression
0,0,previously seen hyperintensities flare sequenc...,small foci of bilateral calcification within t...
1,1,stable cardiomegaly without evidence pulmonary...,picc line terminates at mid svc without eviden...
2,2,heart stably enlarged patient status post cabg...,stable cardiomegaly minimal if any cardiac fai...
3,3,stable cardiomegaly minimal upper lung zone re...,no evidence of pneumonia stable cardiomegaly a...
4,4,tip right sided picc line seen overlying proxi...,tip of rightsided picc line overlying the prox...
...,...,...,...
76139,76139,suspicious liver lesions identified biliary di...,no focal liver lesion identified cholelithiasi...
76140,76140,grayscale color doppler images obtained right ...,no evidence of deep vein thrombosis in the rig...
76141,76141,portable chest radiograph demonstrates focal c...,no acute cardiopulmonary process
76142,76142,ap upright lateral chest radiograph obtained l...,no acute intrathoracic process with left later...


In [61]:
import regex as re
def detect_over(x):
    #starting_position=[x1.start(),x1.end() for x1 in re.finditer(r'(over)',x)]
    for x in re.finditer(r'(over):',x):
        if x.start()==None:
            return 'shit'
        else:
            return(x.start(),x.end())
data_for_training["whether_exist"]=data_for_training['findings'].apply(detect_over)

In [63]:
data_for_training.drop(columns={'Unnamed: 0','whether_exist'},inplace=True)

In [65]:
data_for_training.isna().sum()

findings       0
impression    62
dtype: int64

In [66]:
data_for_training.dropna(inplace=True)

In [67]:
data_for_training.isna().sum()

findings      0
impression    0
dtype: int64

In [68]:
#data_for_training[data_for_training['whether_exist']=='shit']
for i in list(data_for_training['impression']):
    v1=re.findall(r'/(over/)',i)
    if v1 ==[]:
        pass
    else:
        print('over is present')

In [71]:
list(data_for_training['findings'])[1]

'stable cardiomegaly without evidence pulmonary vascular redistribution mediastinal hilar contours normal rightsided duallumen catheter unchanged position tip mid svc new rightsided picc line extends mid svc well pneumothoraces identified lungs remain clear pleural effusions'

In [84]:
data_for_training.drop(columns={'length_check'},inplace=True)
data_for_training["length_check"]=data_for_training['impression'].apply(lambda x : len(x.split(' ')))

In [94]:
index_to_drop=data_for_training[data_for_training['length_check']<=2].index

In [98]:
data_for_training.drop(labels=index_to_drop,axis=0,inplace=True)

In [100]:
#data_for_training.drop(columns={'length_check'},inplace=True)

In [103]:
data_for_training.to_csv('C://Users//kk//Downloads//cleaned_but_need_improvement.csv')